# Connecting to Drive

In [2]:
# Connecting to google drive
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# Path
%cd "/content/drive/MyDrive/Bridge construction health/brridge-failure-detection"

/content/drive/MyDrive/Bridge construction health/brridge-failure-detection


In [5]:
! ls -ltrh

total 4.6M
-rw------- 1 root root 868K Aug 19 15:51 data.rar
-rw------- 1 root root 3.4M Aug 19 15:51 data1.rar
-rw------- 1 root root 373K Aug 21 16:26 wavelet.ipynb
-rw------- 1 root root 3.0K Aug 21 16:26 sst_nino3.dat
-rw------- 1 root root  35K Aug 21 16:26 analysis.ipynb
-rw------- 1 root root 8.0K Aug 21 16:29 github_handler.ipynb
-rw------- 1 root root 1.2K Aug 21 16:30 untitiled.ipynb


# Imports and installations

In [25]:
import os
import shutil
from pathlib import Path

# Unzipping the dataset of CWT images

In [ ]:
! unrar x data.rar ./

In [ ]:
! unrar x data1.rar ./data1